In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv("contest_train_binary.csv")
test_df = pd.read_csv("contest_test.csv")

In [ ]:
#Убираем те столбцы, которые не несут никакой информации
#Срабатывает со второго раза
for i in range(len(df.columns)):
  if (df.iloc[:,i:i+1].value_counts().count() == 1):
    df = df.drop(df.columns[i], axis = 1)


In [ ]:
#Убираем те столбцы, которые не несут никакой информации
#Срабатывает со второго раза
for i in range(len(test_df.columns)):
  if (test_df.iloc[:,i:i+1].value_counts().count() == 1):
    test_df = test_df.drop(test_df.columns[i], axis = 1)

In [ ]:
#Есть разница в количестве убранных столбцов, убираем из теста вот этот, которого нет в train
test_df.drop("FEATURE_256", axis=1, inplace=True)

In [ ]:
#Попробуем перевести в логичекую переменную, но по сути это ничего не изменит
df["TARGET"] = df["TARGET"].astype(bool)

In [ ]:
#Подготовим фрейм для сабмита
result = pd.DataFrame(test_df["ID"])
test_df.drop("ID", axis=1, inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#Заполняем nan
df = df.apply(lambda x: x.fillna(x.median()))
test_df = test_df.apply(lambda x: x.fillna(x.median()))

In [ ]:
#Делим train на свой train и test
Y = df["TARGET"]
X = df.drop(["TARGET", "ID"], axis = 1)
X = (X - X.mean())/X.std()
test_df = (test_df - test_df.mean())/test_df.std()
#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, shuffle=False)

In [ ]:
from sklearn.naive_bayes import *

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.metrics import plot_confusion_matrix, roc_auc_score

In [ ]:
#Попытка сделать Байеса
bay = GaussianNB()
bay.fit(X, Y)
prediction = bay.predict_proba(test_df)
result["TARGET"] = prediction[:,1]
result.to_csv('result1.csv', encoding='utf-8', index=False)

In [ ]:
from sklearn.linear_model import *
from sklearn.tree import *

In [ ]:
#Попробем побить и найти ту часть, где модель натренируется лучше
import sklearn.model_selection

classif_bay = GaussianNB()
logreg = LogisticRegression(solver = 'liblinear')
destree = DecisionTreeClassifier(max_depth = 8)

kf = sklearn.model_selection.KFold(n_splits=10, shuffle=False)
scores_bay = []
data_res = []
roc_auc_pred_proba = {}
roc_auc_logreg = {}
roc_auc_destree = {}

for train_index, test_index in kf.split(X):
    X_train, X_test = X.values[train_index], X.values[test_index]
    y_train, y_test = Y.values[train_index], Y.values[test_index]

    classif_bay.fit(X_train, y_train)
    logreg.fit(X_train, y_train)
    destree.fit(X_train, y_train)

    pred_bay = classif_bay.predict(X_test)
    pred_proba = classif_bay.predict_proba(X_test)

    pred_logreg = logreg.predict(X_test)
    pred_logreg_proba = logreg.predict_proba(X_test)

    pred_destree = destree.predict(X_test)
    pred_destree_proba = destree.predict_proba(X_test)
#Кладём по roc-auc нужные train и test, чтобы потом опять провести то же уже с данными с полного test-а
    #scores_bay.append(roc_auc_score(y_test, pred_bay))
    roc_auc_pred_proba[roc_auc_score(y_test, pred_bay)] = (X_train, y_train, pred_proba, X_test)
    roc_auc_logreg[roc_auc_score(y_test, pred_logreg)] = (X_train, y_train, pred_logreg_proba)
    roc_auc_destree[roc_auc_score(y_test, pred_destree)] = (X_train, y_train, pred_destree_proba)
    
print(max(roc_auc_pred_proba.keys()))    
roc_auc_pred_proba.get(max(roc_auc_pred_proba.keys()))

0.5868377048170799


(array([[-0.23657309,  0.36255069, -0.24447774, ..., -0.37088072,
          1.99514897,  2.03171264],
        [-0.30538043, -0.18497074, -0.24447774, ..., -0.37088072,
         -0.4552917 , -0.46026543],
        [-0.37418776, -0.73249217, -0.24447774, ..., -0.37088072,
         -0.4552917 , -0.46026543],
        ...,
        [-0.37418776, -0.73249217, -0.24447774, ..., -0.37088072,
         -0.4552917 , -0.46026543],
        [-0.16776576,  0.36255069, -0.24447774, ..., -0.37088072,
          1.99514897, -0.46026543],
        [-0.23657309,  0.36255069, -0.24447774, ..., -0.37088072,
          1.99514897, -0.46026543]]),
 array([ True, False,  True, ..., False,  True, False]),
 array([[6.98366614e-01, 3.01633386e-01],
        [9.37915545e-01, 6.20844553e-02],
        [9.99927305e-01, 7.26951086e-05],
        ...,
        [3.05766756e-03, 9.96942332e-01],
        [1.52260565e-01, 8.47739435e-01],
        [9.71447447e-01, 2.85525526e-02]]),
 array([[-0.37418776, -0.73249217, -0.24447774, .

In [ ]:
list_x_tr_y_tr = roc_auc_pred_proba.get(max(roc_auc_pred_proba.keys()))
bayes = GaussianNB()
bayes.fit(list_x_tr_y_tr[0], list_x_tr_y_tr[1])
prediction_proba = bayes.predict_proba(test_df)
prediction_proba[:,1]

array([9.93499283e-01, 6.11919392e-01, 2.26837424e-05, ...,
       2.60902706e-01, 9.97479308e-01, 9.99377377e-01])

In [ ]:
result["TARGET"] = prediction_proba[:,1]
result.to_csv('result2.csv', encoding='utf-8', index=False)

In [ ]:
list_logreg = roc_auc_logreg.get(max(roc_auc_logreg.keys()))
logreg1 = LogisticRegression(solver = 'liblinear')
logreg.fit(list_logreg[0], list_logreg[1])
pred_logreg = logreg.predict_proba(test_df)
print(pred_logreg)

list_destree = roc_auc_destree.get(max(roc_auc_destree.keys()))
destree1 = DecisionTreeClassifier(max_depth = 8)
destree.fit(list_destree[0], list_destree[1])
pred_destree = destree.predict_proba(test_df)
print(pred_destree)

[[0.76933761 0.23066239]
 [0.84719347 0.15280653]
 [0.88173703 0.11826297]
 ...
 [0.95648223 0.04351777]
 [0.83852756 0.16147244]
 [0.81107834 0.18892166]]
[[0.14285714 0.85714286]
 [0.63111111 0.36888889]
 [0.91957758 0.08042242]
 ...
 [0.82051282 0.17948718]
 [0.63808901 0.36191099]
 [0.74160207 0.25839793]]


In [ ]:
roc_auc_destree.keys()

dict_keys([0.5063894544535125, 0.508209101798395, 0.5194988509155608, 0.5029962839020793, 0.5149480548187444, 0.5208166776859048, 0.5106113161072438, 0.5083513735001398, 0.5095782761899434, 0.5214975475168608])

In [ ]:
result["TARGET"] = pred_logreg[:,1]
result.to_csv('result_logreg.csv', encoding='utf-8', index=False)

In [ ]:
result["TARGET"] = pred_destree[:,1]
result.to_csv('result_destree.csv', encoding='utf-8', index=False)

Итоговый максимальный результат дало дерево - 0.51153 (результат уже с кагла)